In [17]:
import hashlib
import pandas as pd

raw_csv = pd.read_csv('raw_data.csv')
max_colname_len = max(
    [len(col) for col in raw_csv.columns.values]
)
for col in raw_csv.columns.values:
    print('{:{}}<{}>'.format(
        col, max_colname_len+2, raw_csv[col].dtype)
    )

Bird Species  <object>
Distance Bin  <object>
Date          <object>
Time          <object>
Coordinates   <object>
Location      <object>
Habitat Type  <object>
Time Period   <object>
Group Name    <object>
Remarks       <object>


In [18]:
num_locations = len(set(raw_csv['Location']))
num_coords = len(set(raw_csv['Coordinates']))
print('Number of Unique Locations: {}'.format(num_locations))
print('Number of Unique Coordinates: {}'.format(num_coords))

Number of Unique Locations: 208
Number of Unique Coordinates: 271


First of all, we need to be able to clearly distinguish survey sites. As from the cell directly above, we cannot reliably use either the number of unique locations or coordinates in order to do this.

Instead, we will use both. Going through the list from the first to last row, _adjacent_ rows with identical locations **or** coordinate values will be grouped together as one survey site. Each survey site is identifed with a hash generated from the cocatenated string of its first row's location plus coordinate.

In [19]:
csv_unique_surveys = pd.DataFrame()
lagging_pointer = {'Location': None, 'Coordinates': None}

for _, row in raw_csv.iterrows():
    ...